# Using LHS Value to Debug An Infeasible Problem

This notebook aims to show how to use the left-hand side (LHS) value (`Constraint.e`) to debug an infeasible problem.

The LHS value is the value of the left-hand side of the equation. It is useful to check which constraints are violated.

In [1]:
import ams

In [2]:
ams.config_logger(stream_level=20)

First, let's solve a DCOPF.

In [3]:
sp = ams.load(ams.get_case('matpower/case14.m'),
              no_output=True)

Parsing input file "/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/ams/cases/matpower/case14.m"...
Input file parsed in 0.0028 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
System set up in 0.0014 seconds.


In [4]:
sp.DCOPF.run(solver='CLARABEL')

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.0091 seconds.
<DCOPF> solved as optimal in 0.0080 seconds, converged in 11 iterations with CLARABEL.


True

In [5]:
sp.DCOPF.pg.v.round(2)

array([2.21, 0.38, 0.  , 0.  , 0.  ])

Then, we can lower down the generator maximum output to create an infeasible problem.

In [6]:
sp.StaticGen.set(src='pmax', attr='v', idx=[1, 2, 3, 4, 5], value=0.5)

True

Remember to update the corresponding parameters in the routine.

In [7]:
sp.DCOPF.update('pmax')

True

Now, we can see that the problem is infeasible.

In [8]:
sp.DCOPF.run(solver='CLARABEL')

DCOPF failed as infeasible in 9 iterations with CLARABEL!


False

Then, we can turn off some constraints and sovle it again.

In [9]:
sp.DCOPF.disable(['pglb', 'pgub'])

Turn off constraints: pglb, pgub


True

Note that since there are some constraints touched, the OModel will be updated automatically.

In [10]:
sp.DCOPF.run(solver='CLARABEL')

Disabled constraints: pglb, pgub
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.0014 seconds.
<DCOPF> solved as optimal in 0.0050 seconds, converged in 9 iterations with CLARABEL.


True

In [11]:
sp.DCOPF.pg.v

array([ 2.31448356,  0.39836206, -0.04094854, -0.04094854, -0.04094854])

Since all the inequality constraints are written in the form of $Ax -b \leq 0$, we can check the violated constraints by finding the those with positive `Constraint.e`.

In [12]:
sp.DCOPF.pgub.e

array([ 1.81448356, -0.10163794, -0.54094854, -0.54094854, -0.54094854])

In [13]:
sp.DCOPF.pglb.e

array([-2.31448356, -0.39836206,  0.04094854,  0.04094854,  0.04094854])